## Goal
The objective of this project is to build one or more regression models to determine the scores for each team using the other columns as features. I will follow data preparation and feature engineering process before I use a regression model to predict the scores.

In [128]:
import pandas as pd #load the data into panda data frames
from sklearn import preprocessing #label encoder
from sklearn.model_selection import train_test_split #split the dataset

## Load the dataset
I am using the [soccer dataset](https://github.com/fivethirtyeight/data/tree/master/soccer-spi) from FiveThirtyEight.


When I viewed the full dataset, I noticed that some rows' scores are not provided. These games have not taken place yet and hence we only have access to their Soccer Power Index (SPI), which [ESPN describes](https://www.espn.com/soccer/news/story/_/id/1873765) as the "best possible representation of a team's current overall skill level". An inconsistency in some rows having or not having score values would not be good to maintain a uniform format.

The first few entries in past dates have scores.

In [129]:
soccer_stats = pd.read_csv('../data/raw/spi_matches.csv')
soccer_stats.head(2)

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


While future dates in 2021 do not have score data.

In [130]:
soccer_stats.tail(2)

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
42172,2020,2021-05-30,1871,Spanish Segunda Division,Logrones,Las Palmas,29.82,36.56,0.352,0.3751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42173,2020,2021-05-30,1871,Spanish Segunda Division,Mirandes,CD Sabadell,31.03,30.30,0.435,0.2938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
soccer_stats.keys()

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object')

I understood what the columns meant through some research on an [rdrr.io dataset page](https://rdrr.io/github/fivethirtyeightdata/fivethirtyeightdata/man/spi_matches.html) 

- **season:** Season of the soccer game.

- **date:** The date that the match took place.

- **league_id:** A numerical identifier of the league within which the match was played.

- **league:** League name.

- **team1:** One team that participated in the match.

- **team2:** The other team that participated in the match.

- **spi1:** The SPI score of team1.

- **spi2:** The SPI score of team2.

- **prob1:** The probability that team1 would have won the match.

- **prob2:** The probability that team2 would have won the match.

- **probtie:** The probability that the match would have resulted in a tie.

- **proj_score1:** The predicted number of goals that team1 would have scored.

- **proj_score2:** The predicted number of goals that team2 would have scored.

- **score1:** The number of goals that team1 scored.

- **score2:** The number of goals that team2 scored.

The following columns did not have any description importance1, importance2, xg1, xg2, nsxg1, nsxg2, adj_score1, adj_score2 and seem to have NaN values for some rows.

## Data Preparation
###  Cleaning

1. Drop all rows with NaN values. These include rows without scores, as well as any other unknown values.
2. Remove columns without descriptions. Since we do not have enough information  about these columns, we cannot make educated decisions about how it affects the dataset. So, we drop them:

In [132]:
soccer_stats = soccer_stats.dropna()
soccer_stats = soccer_stats.drop(columns=['importance1', 'importance2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2']) 

To double check that we have removed all rows with a NaN score, I display all rows with NaN values.

In [133]:
soccer_stats[soccer_stats.isna().any(axis=1)]

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2


..and nothing is returned. So, we have sucessfully cleaned our data! 

This is how it looks:

In [134]:
soccer_stats.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2
10,2016,2016-08-12,1843,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,0.1157,0.91,2.36,0.0,1.0
11,2016,2016-08-12,1843,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,0.2617,1.82,0.86,2.0,2.0
12,2016,2016-08-13,2411,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,0.2921,1.16,1.24,2.0,1.0
13,2016,2016-08-13,2411,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,0.2689,1.47,1.38,1.0,1.0
14,2016,2016-08-13,2411,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,0.2847,1.35,1.14,0.0,1.0


## Feature Engineering
### Extraction
I separate the date column into three date, month and year. The season and year seem to not always align because a season may span between two years.

In [135]:
soccer_stats[['Year','Month', 'Date']] = soccer_stats.date.str.split("-",expand=True)
soccer_stats = soccer_stats.drop(columns=['date'])
soccer_stats.head()

,season,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2,Year,Month,Date
10,2016,1843,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,0.1157,0.91,2.36,0.0,1.0,2016,08,12
11,2016,1843,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,0.2617,1.82,0.86,2.0,2.0,2016,08,12
12,2016,2411,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,0.2921,1.16,1.24,2.0,1.0,2016,08,13
13,2016,2411,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,0.2689,1.47,1.38,1.0,1.0,2016,08,13
14,2016,2411,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,0.2847,1.35,1.14,0.0,1.0,2016,08,13


### Transformation
To perform statistical analysis and to pass the data into the regression models, I encode the strings into integer values as the models do not work with strings.

**League Name**

In [136]:
league_encode = preprocessing.LabelEncoder()
league_encode.fit(soccer_stats['league'])
soccer_stats['league'] = league_encode.transform(soccer_stats['league'])

**Team names**

In [137]:
team1_encode = preprocessing.LabelEncoder()
team1_encode.fit(soccer_stats['team1'])
soccer_stats['team1'] = team1_encode.transform(soccer_stats['team1'])

team2_encode = preprocessing.LabelEncoder()
team2_encode.fit(soccer_stats['team2'])
soccer_stats['team2'] = team2_encode.transform(soccer_stats['team2'])

Encoded data looks like this:

In [138]:
soccer_stats.head()

,season,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2,Year,Month,Date
10,2016,1843,7,48,294,51.16,85.68,0.0463,0.8380,0.1157,0.91,2.36,0.0,1.0,2016,08,12
11,2016,1843,7,10,191,68.85,56.48,0.5714,0.1669,0.2617,1.82,0.86,2.0,2.0,2016,08,12
12,2016,2411,2,205,227,53.57,66.81,0.3459,0.3621,0.2921,1.16,1.24,2.0,1.0,2016,08,13
13,2016,2411,2,136,396,68.02,73.25,0.3910,0.3401,0.2689,1.47,1.38,1.0,1.0,2016,08,13
14,2016,2411,2,115,428,55.19,58.66,0.4214,0.2939,0.2847,1.35,1.14,0.0,1.0,2016,08,13


## Regression Models


In [139]:
classes = soccer_stats[['score1', 'score2']]
features = soccer_stats[['season', 'league_id', 'league', 'team1', 'team2', 'spi1', 'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'Year', 'Month', 'Date']]

In [140]:
X_train, X_test, Y_train, Y_test = train_test_split(features, classes, test_size=0.2, random_state=1) 